In [1]:
import googlemaps
import os
from langchain.agents import create_agent
from langchain.tools import tool
from langchain_google_genai import ChatGoogleGenerativeAI
from typing import List, Literal, Optional
from tavily import TavilyClient
from pydantic import BaseModel, Field
from serpapi import GoogleSearch

In [ ]:
os.environ["GOOGLE_API_KEY"] = ""
os.environ["TAVILY_API_KEY"] = ""
os.environ["http_proxy"] = "http://127.0.0.1:7890"
os.environ["https_proxy"] = "http://127.0.0.1:7890"
tavily_api = os.environ["TAVILY_API_KEY"]
model = ChatGoogleGenerativeAI(model="gemini-2.5-flash", temperature=0, max_retries = 3)
tavily_client = TavilyClient(tavily_api)

In [3]:
@tool("Web_Search_Tool", description = " This tool is used when there is a need to search online.")
def web_search_tool(
    query: str, 
    country: str = "us", 
    max_results: int = 5,
    search_depth: Literal["basic", "advanced"] = "advanced",
    topic: Literal["general", "news"] = "general"
):
    """
    Perform an advanced web search using the Tavily API to find real-time information, 
    news, or travel itineraries.
    
    Use this tool when you need to find:
    - Current events or news (set topic="news")
    - Travel itineraries, guides, and blogs
    - Facts that are not in your internal knowledge base
    
    Args:
        query (str): The search query. Be specific (e.g., "3 day itinerary for Tokyo" instead of "Tokyo").
        country (str): The 2-letter ISO country code to bias results towards (e.g., "us", "fr", "jp"). Defaults to "us".
        max_results (int): The maximum number of search results to return. Defaults to 5.
        search_depth (str): "basic" for fast results, "advanced" for high-quality content. Defaults to "advanced".
        topic (str): "general" for most queries, "news" for recent events. Defaults to "general".

    Returns:
        list[dict]: A list of dictionaries containing 'url', 'content', and 'title' for each result.
    """
        
    try:
        if topic == "general":
            response = tavily_client.search(
                query=query,
                search_depth=search_depth,
                topic=topic,
                country=country,
                max_results=max_results
            )
        else:
            response = tavily_client.search(
                query=query,
                search_depth=search_depth,
                topic=topic,
                max_results=max_results
            )
        return response.get("results", [])

    except Exception as e:
        return f"Error performing search: {str(e)}"


In [ ]:
web_search_system_prompt = """WebSearchAgent: Search the web for real-time information.

TOOL: web_search_tool(query, max_results, search_depth, topic)

PARAMETERS:
- query: Specific search query
- max_results: 3-5 recommended
- search_depth: Always "advanced"
- topic: "news" for this week's events, "general" for everything else

SEARCH STRATEGY:
1. For policies, visa rules, or guides: START with topic="general"
2. For breaking news or events: Use topic="news"
3. If results are irrelevant, try rephrasing the query or switching topic

EXAMPLES:
- "Pakistan visa policy" → topic="general" (policy info, not breaking news)
- "Pakistan earthquake today" → topic="news" (current event)
- "Tokyo 3-day itinerary" → topic="general" (travel guide)

OUTPUT:
**Search Results: {query}**

1. [{Title}]({URL}) - {Key info}
2. [{Title}]({URL}) - {Key info}

**Summary:** {Brief synthesis}

If results don't match the query, try a different search or report no relevant info found."""

web_search_agent = create_agent(model = model, system_prompt = web_search_system_prompt , tools = [web_search_tool])

In [ ]:
ans = web_search_agent.invoke({"messages": "Tell me about new tourist visa poilicy of China"})
print(ans['messages'][-1])

content=[{'type': 'text', 'text': 'China has implemented several new tourist visa policies:\n\n*   **Norway Visa-Free:** Effective September 9, 2024, Norwegian citizens can visit China for up to 15 days without a visa.\n*   **Cruise Ship Visa-Free:** As of May 15, 2024, foreign tourist groups arriving by cruise ship at any Chinese coastal port can enter visa-free for up to 15 days. They must remain with the same cruise ship and within designated areas.\n*   **Hainan Island Visa-Free:** Since May 1, 2018, citizens from 59 countries can visit Hainan Island for up to 30 days without a visa. This policy allows for individual tourism, though tourists still need to inform a tour agency.\n*   **Unilateral Visa-Free Extension:** China has extended a unilateral visa-free policy for ordinary passport holders from several European countries (including France, Germany, Italy, the Netherlands, Spain, Switzerland, Ireland, and Hungary). This allows for visa-free stays of up to 30 days for purposes s